# REFINITIV COMPANY TEARSHEET

In [194]:
RIC = input()
# print(RIC)

IBM


In [260]:
# Import module and set up with App Key
import eikon as ek
ek.set_app_key('8a0a51d096f34eec86dadab1763ad94dae81c80e')
import pandas as pd 

In [261]:
# Request Header Information for RIC 
df, err = ek.get_data(RIC, "TR.CommonName(),TR.HeadquartersCountry,TR.TRBCIndustryGroup,TR.GICSSector,TR.CompanyMarketCap.Currency")
#Request Exchange for Primary Quote RIC
df_2, err = ek.get_data(RIC,"TR.PrimaryQuote")
df_3, err = ek.get_data(df_2.at[0,'Primary Quote RIC'],"TR.ExchangeName")  
df['Exchange Name'] = [df_3.at[0,'Exchange Name']]
df

,Instrument,Company Common Name,Country of Headquarters,TRBC Industry Group Name,GICS Sector Name,Currency,Exchange Name
0,IBM,International Business Machines Corp,United States of America,Software & IT Services,Information Technology,USD,"NEW YORK STOCK EXCHANGE, INC."


## General Information

In [159]:
dfGen, err = ek.get_data(RIC, 'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian',{'Curn':df.at[0,'Currency'],'Transpose':'y'})
dfGen2, err = ek.get_data( RIC, 'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV',{'Scale':6,'Curn':df.at[0,'Currency'],'transpose':'y'})
display(dfGen,dfGen2)

,Instrument,Price Close,Price 52 Week High,52-week High Price PCT Change,Price 52 Week Low,52-week Low Price PCT Change,Price Target - Median
0,IBM,102.81,158.75,-35.237795,102.28,0.518185,152.5


,Instrument,Beta 5 Year,Dividend yield,Outstanding Shares,Free Float,Free Float (Percent),Company Market Cap,Enterprise Value (Daily Time Series)
0,IBM,1.406707,5.492922,888.408023,887.410463,99.8877,91337.228845,145512.228845


## Consensus Recommendation

In [160]:
df6, err = ek.get_data(RIC, 'TR.RecLabel();TR.NumOfRecommendations();TR.NumEstRevisingUp;TR.NumEstRevisingDown',{'WP':'30d','EstimateMeasure':'EPS'})
df6

,Instrument,Recommendation - Mean Label,Recommendation - Number Of Total,Number of Analysts Revising Up,Number of Analysts Revising Down
0,IBM,HOLD,20,1,1


## Estimates

In [161]:
#Retrieve Estimates into Data Frames per Period

#FY0
dfFY0, err = ek.get_data(RIC, 'TR.RevenueActValue;TR.EBITDAActValue;TR.EPSActValue',
                       {'Period':'FY0', 'Scale':'6', 'Curn':df.at[0,'Currency']})
#FY1
dfFY1, err = ek.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY1', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY1_2, err = ek.get_data(RIC, 
                           'TR.RevenueMeanEstimate; TR.EBITDAMean; TR.EPSMeanEstimate',
    {'Period':'FY1','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})
#FY2
dfFY2, err = ek.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY2', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY2_2, err = ek.get_data(RIC, 'TR.RevenueMeanEstimate;TR.EBITDAMean;TR.EPSMeanEstimate',
    {'Period':'FY2','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})

### DEBUG display(dfFY0,dfFY1,dfFY1_2, dfFY2, dfFY2_2)

In [162]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# initialise results per column 
data = {'Sales (MIL) Mean EST':
        [dfFY0.at[0,'Revenue - Actual'], dfFY1.at[0,'Revenue - Mean Estimate'],'',dfFY2.at[0,'Revenue - Mean Estimate'],''],
       'High Est':
       ['',dfFY1.at[0,'Revenue - High'],(dfFY1.at[0,'Revenue - High']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - High'],(dfFY2.at[0,'Revenue - High']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Low Est':
       ['',dfFY1.at[0,'Revenue - Low'],(dfFY1.at[0,'Revenue - Low']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - Low'],(dfFY2.at[0,'Revenue - Low']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Est. 30 30 Days Ago':
       ['',dfFY1_2.at[0,'Revenue - Mean Estimate'],(dfFY1_2.at[0,'Revenue - Mean Estimate']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Revenue - Mean Estimate'],(dfFY2_2.at[0,'Revenue - Mean Estimate']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'EBITDA (MIL) Mean Est.':
        [dfFY0.at[0,'EBITDA - Actual'],dfFY1.at[0,'EBITDA - Mean'],'',dfFY2.at[0,'EBITDA - Mean'],''],
        'High Est.':
        ['',dfFY1.at[0,'EBITDA - High'],(dfFY1.at[0,'EBITDA - High']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - High'],(dfFY2.at[0,'EBITDA - High']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Low Est.':
        ['',dfFY1.at[0,'EBITDA - Low'],(dfFY1.at[0,'EBITDA - Low']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - Low'],(dfFY2.at[0,'EBITDA - Low']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Est. 30 30 Days Ago.':
       ['',dfFY1_2.at[0,'EBITDA - Mean'],(dfFY1_2.at[0,'EBITDA - Mean']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
        dfFY2_2.at[0,'EBITDA - Mean'],(dfFY2_2.at[0,'EBITDA - Mean']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'EPS Mean Est.':
        [dfFY0.at[0,'Earnings Per Share - Actual'], dfFY1.at[0,'Earnings Per Share - Mean Estimate'],'',
         dfFY2.at[0,'Earnings Per Share - Mean Estimate'],''],
        'High Est..':
         ['',dfFY1.at[0,'Earnings Per Share - High'],(dfFY1.at[0,'Earnings Per Share - High']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - High'],(dfFY2.at[0,'Earnings Per Share - High']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Low Est..':
         ['',dfFY1.at[0,'Earnings Per Share - Low'],(dfFY1.at[0,'Earnings Per Share - Low']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - Low'],(dfFY2.at[0,'Earnings Per Share - Low']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Est. 30 30 Days Ago..':
        ['',dfFY1_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY1_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY2_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100]
       }  
  
# Create andas DataFrame. 
dfMultiRow = pd.DataFrame(data, index =['FY0 (Actual)','FY1 (Current Year)', '% above/below mean','FY2 (Next Year)','% above/below mean' ]) 
                                             
dfMultiRow                     

,Sales (MIL) Mean EST,High Est,Low Est,Est. 30 30 Days Ago,EBITDA (MIL) Mean Est.,High Est.,Low Est.,Est. 30 30 Days Ago.,EPS Mean Est.,High Est..,Low Est..,Est. 30 30 Days Ago..
FY0 (Actual),77147,,,,19029,,,,12.81,,,
FY1 (Current Year),78673.9,80600,75524,78837.7,18946.7,21202,16779,19005.3,13.3325,13.62,12.5,13.3715
% above/below mean,,2.44816,-4.00378,0.208212,,11.9034,-11.4411,0.309289,,2.15623,-6.24428,0.292368
FY2 (Next Year),79982.3,83400,75661,80159.2,19958,21997,17034,20015.7,14.152,14.7,13.5,14.1767
% above/below mean,,4.27303,-5.40285,0.221179,,10.2165,-14.6507,0.289107,,3.87188,-4.60746,0.17425


# Valuation Ratios

In [163]:
# Retrieve Valuation Ratios into DataFrames per period

dfLTM, err  = ek.get_data(RIC,'TR.PE;TR.PriceToSalesPerShare;TR.EVToSales;TR.EVToEBITDA',{'Sdate':'0D'})
dfFY1, err = ek.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY1'})
dfFY1_2, err= ek.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY1','Curn':df.at[0,'Currency']})
dfFY2, err = ek.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY2'})
dfFY2_2, err= ek.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY2','Curn':df.at[0,'Currency']})
###DEBUG display(dfLTM,dfFY1,dfFY1_2,dfFY2, dfFY2_2)

In [164]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# Initialise results per column 
dataDict = {'P/E':
        [dfLTM.at[0,'P/E (Daily Time Series Ratio)'], dfFY1.at[0,'Price / EPS (Mean Estimate)'], dfFY2.at[0,'Price / EPS (Mean Estimate)']],
            'P/SALES':
        [dfLTM.at[0,'Price To Sales Per Share (Daily Time Series Ratio)'], dfFY1.at[0,'Price / Revenue (Mean Estimate)'], dfFY2.at[0,'Price / Revenue (Mean Estimate)']],
            'EV/REVS':
                [dfLTM.at[0,'Enterprise Value To Sales (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'Revenue - Mean Estimate'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'Revenue - Mean Estimate']],
            
            'EV/EBITDA':
                [dfLTM.at[0,'Enterprise Value To EBITDA (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'EBITDA - Mean'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'EBITDA - Mean']]
}

#Create Pandas DataFrame. 
dfMultiRow = pd.DataFrame(dataDict, index =['LTM (Last Twelve Months)','FY1 (Current Year)', 'FY2 (Next Year)']) 
dfMultiRow        

,P/E,P/SALES,EV/REVS,EV/EBITDA
LTM (Last Twelve Months),10.994388,1.358499,2.060721,8.269897
FY1 (Current Year),7.711220,1.160959,1.849561,7.680083
FY2 (Next Year),7.264672,1.141968,1.819305,7.290928


## FY0 Per Share Metrics 

In [165]:
dfFYOShare, err = ek.get_data(RIC ,['TR.TotCashAndShortTermInvestmentsperShare',
'TR.WorkingCapitalperShare','BookValuePerShare','TR.RevenuePerShare','TR.CashFlowtoShrdil',
'TR.FreeOperatingCashFlowperShareAvgDilutedSharesOut','TR.DPSCommonStock'], {'Period':'FY0','curn':df.at[0,'Currency']})
dfFYOShare_2, err =  ek.get_data(RIC ,['TR.TotalDebt','TR.TtlCmnSharesOut'],{'Period':'FY0','curn':df.at[0,'Currency']})
# additionally, calculate total debt
dfFYOShare['Total Debt'] = [dfFYOShare_2.at[0,'Total Debt']/dfFYOShare_2.at[0,'Total Common Shares Outstanding']]
dfFYOShare

,Instrument,Total Cash & ST Invest/Share,Wk Cap/Shr,BOOKVALUEPERSHARE,Revenue Per Share,Cash Flow/Shr (dil.),FOCF/Shr (dil.),DPS - Common Stock Primary Issue,Total Debt
0,IBM,9.9965,0.810497,NaN,86.408875,17.517659,6.896178,6.43,70.903234


## FY0 Employee Data

In [166]:
dfEmp, err = ek.get_data(RIC,'TR.Employees(Period=FY0),TR.RevenueperEmployeescalYrEnd(Period=FY0)')
dfEmp_2, err = ek.get_data(RIC,'TR.Employees(Period=FY-1)')
dfEmp_3, err = ek.get_data(RIC,'TR.RevenueperEmployeescalYrEnd(Period=FY0,sdate=0D,Edate=-4FY,Frq=FY)')
dfEmp['Empt Grth'] = (dfEmp_2.at[0,'Full-Time Employees']-dfEmp.at[0,'Full-Time Employees'])/dfEmp_2.at[0,'Full-Time Employees']
dfEmp['5 yr Avg'] = dfEmp_3["Revenue/Employee, FY"].mean()
dfEmp

,Instrument,Full-Time Employees,"Revenue/Employee, FY",Empt Grth,5 yr Avg
0,IBM,352600,219416.951081,-0.005705,215998.501851


## Abbreviated Financials

### Income Statement (MIL)

In [167]:
dfFin, err = ek.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency'],'Transpose':'Y'})
dfFin2, err = ek.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency'],'Transpose':'Y'})

# results as simple lists
listFin = list(dfFin.iloc[0, :])
listFin2 = list(dfFin2.iloc[0, :])

# results as single dictionary
dataDict2 = {'FY0': listFin, 'FY-1': listFin2 }

# Pandas DataFrame from dictionary 
dfIncome = pd.DataFrame(dataDict2, index = dfFin.columns) 
dfIncome        

,FY0,FY-1
Instrument,IBM,IBM
Total Revenue,77147,79591
Gross Profit,36488,36936
EBITDA,18986,17725
EBIT,12927,13245
Net Income Before Taxes,10166,11342
Net Income Incl Extra Before Distributions,9431,8728
Total Common Shares Outstanding,887.11,892.479


### Balance Sheet (MIL)

In [168]:
dfBal, err = ek.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency'],'Transpose':'Y'})
dfBal2, err = ek.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency'],'Transpose':'Y'})

# results as single dictionary
dataDict3 = {'FY0': list(dfBal.iloc[0, :]), 'FY-1': list(dfBal2.iloc[0, :]) }

# Pandas DataFrame from dictionary 
dfBalance= pd.DataFrame(dataDict3, index = dfBal.columns) 
dfBalance 

,FY0,FY-1
Instrument,IBM,IBM
Cash and Short Term Investments,8868,11999
"Property/Plant/Equipment, Total - Net",15006,10792
"Total Assets, Reported",152186,123382
Current Liabilities,37701,38227
Total Long Term Debt,54102,35605
Total Liabilities,131345,106587
Total Equity,20841,16795


### Cash Flow Statement (MIL)

In [169]:
dfCash, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency']})
dfCash_2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY0','Scale':'6','Curn':df.at[0,'Currency']})
dfCash_3, err = ek.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency'],'Transpose':'Y'})
#insert as columns, entincome and  sum of cash from operations
dfCash_3['Net Income'] = [dfCash.at[0,'Net Income/Starting Line, Cumulative']]
dfCash_3['Cash From Operations'] = [sum(list(dfCash_2.iloc[0, 1:]))]

dfCash2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency']})
dfCash2_2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY-1','Scale':'6','Curn':df.at[0,'Currency']})
dfCash2_3, err = ek.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency'],'Transpose':'Y'})
#insert as columns, entincome and  sum of cash from operations
dfCash2_3['Net Income'] = [dfCash2.at[0,'Net Income/Starting Line, Cumulative']]
dfCash2_3['Cash From Operations'] = [sum(list(dfCash2_2.iloc[0, 1:]))]

#DEBUG display(dfCash_3,dfCash2_3)

# results as simple lists
listCash = list(dfCash_3.iloc[0, :])
listCash2 = list(dfCash2_3.iloc[0, :])


# results as single dictionary
dataDict3 = {'FY0': listCash, 'FY-1': listCash2 }
#dataDict3

# Pandas DataFrame from dictionary 
dfCashFlow = pd.DataFrame(dataDict3, index = dfCash_3.columns) 
dfCashFlow  

,FY0,FY-1
Instrument,IBM,IBM
"Cash from Operating Activities, Cumulative",14771,15247
"Capital Expenditures, Cumulative",-2907,-3964
"Cash from Investing Activities, Cumulative",-26936,-4913
TR.CASHDIVIDENDSPAID*-1,-5707,-5666
"Cash from Financing Activities, Cumulative",9042,-10469
Net Income,9431,8728
Cash From Operations,13546,14694


## Review of Stock Performance 

### Price Actions

In [201]:
# retrieve percentage changes
dfPriceActions, err = ek.get_data(RIC, 'TR.PricePctChg1M/100;TR.PricePctChg3M/100;TR.PricePctChg6M/100;TR.PricePctChg1Y/100')
dfPriceActions

#create lists out of results 
listNames = list(dfPriceActions.columns)
listPA = list(dfPriceActions.iloc[0, :])

# remove the first element, it's header
listNames.pop(0)
listPA.pop(0)
#display(listNames, listPA)

# Pandas DataFrame from dictionary
dataDictPA = {'Names': listNames, 'Periods': listPA } 
dfPA = pd.DataFrame(dataDictPA, index = listNames) 


import plotly.graph_objects as go
import plotly.express as px
init_notebook_mode(connected=True)

fig = px.bar(dfPA, x='Names', y='Periods')
fig.show()

### Profitability 

In [259]:
dfProfit, err = ek.get_data(RIC, 'TR.GrossMargin;TR.ResearchAndDevelopment;TR.TotalRevenue;TR.SGAExpenseTotal;TR.EBITDAMarginPercent;TR.OperProfitMarginPct;TR.PretaxMarginPercent;TR.NetProfitMargin;TR.CashFlowMarginPercent/100;TR.IncomeTaxRatePct;TR.ROATotalAssetsPercent;TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0','Transpose':'Y'})
#dfProfit   
dfProfit2, err = ek.get_data(RIC, 
    'TR.GrossProfitMarginPct5YrAvg();TR.EBITDAMarginPct5YrAvg;TR.OperatingProfitMarginPct5YrAvg;TR.IncomeBefTaxMarginpct5YrAvg();TR.IncomeAftTaxMarginpct5YrAvg();TR.CashFlowMarginPct5YrAvg()/100;TR.IncomeTaxRatePct5YrAvg()/100;TR.ReturnonAvgTotAssetsPctIncomeAfterTax5YrAvg')
dfProfit3, err = ek.get_data(RIC, 'TR.ResearchAndDevelopment(),TR.TotalRevenue(),TR.SGAExpenseTotal,TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0','Sdate':0,'Edate':-4,'Frq':'FY'})
dfProfit3['RAD over TR'] = dfProfit3['Research And Development']/dfProfit3['Total Revenue']
dfProfit3['SGA over TR'] = dfProfit3['Selling/General/Administrative Expense, Total']/dfProfit3['Total Revenue']
#dfProfit2

#Create a dictionary out of results, and structure a single multi-row Data Frame
# initialise results per column 
dataDictProfit = {'FY0':
        [dfProfit.at[0,'Gross Margin, Percent'],  dfProfit.at[0,'Research And Development']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'Selling/General/Administrative Expense, Total']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'EBITDA Margin, Percent'],dfProfit.at[0,'Operating Profit Margin, Percent'],
        dfProfit.at[0,'Pretax Margin, Percent'],dfProfit.at[0,'Net Profit Margin, (%)'],
        dfProfit.at[0,'TR.CASHFLOWMARGINPERCENT/100'],dfProfit.at[0,'Income Tax Rate, %'],
        dfProfit.at[0,'ROA Total Assets, Percent'],dfProfit.at[0,'ROE Total Equity %']
    ]
        ,'5 Year Avg':[dfProfit2.at[0,'Gross Profit Margin, %, 5 Yr Avg'],dfProfit3['RAD over TR'].mean(),
                       dfProfit3['SGA over TR'].mean(), dfProfit2.at[0,'EBITDA Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Oper Profit Margin, %, 5 Yr Avg'],dfProfit2.at[0,'Income Bef Tax Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Income Aft Tax Margin, %, 5 Yr Avg'],dfProfit2.at[0,'TR.CASHFLOWMARGINPCT5YRAVG()/100'],
                       dfProfit2.at[0,'TR.INCOMETAXRATEPCT5YRAVG()/100'],dfProfit2.at[0,'ROA Tot Assets, %, 5 Yr Avg'],
                       dfProfit3['ROE Total Equity %'].mean()
]
                       
}
#dataDictProfit

# structure as required multi-row
dfMultiProfit = pd.DataFrame(dataDictProfit, index =[
    'Gross Margin','R/D % Sales','SGA % Sales','EBITDA Margin','Operating Margin','Pretax Margin','Net Margin',
    'CF/Sales','Tax Rate','ROA','ROE'
    ]) 
dfMultiProfit   

,FY0,5 Year Avg
Gross Margin,47.296720,47.685373
R/D % Sales,0.077631,0.070337
SGA % Sales,0.252777,0.241042
EBITDA Margin,24.610160,23.801055
Operating Margin,13.177440,15.390266
Pretax Margin,13.177440,15.390517
Net Margin,12.419150,14.292758
CF/Sales,0.202730,0.201579
Tax Rate,5.754476,0.071327
ROA,6.953638,9.292783


### FY0 Sales By Region 

In [265]:
dfSalesByRegion, err = ek.get_data(RIC, 'TR.BGS.GeoTotalRevenue.segmentName;TR.BGS.GeoTotalRevenue()',{'Scale':'6','curn':df.at[0,'Currency']})
dfSalesByRegion

,Instrument,Segment Name,Geographic Total Revenues (Calculated)
0,IBM,Americas,36274
1,IBM,Europe/Middle East/Africa,24443
2,IBM,Asia/Pacific,16430
3,IBM,Segment Total,77147
4,IBM,Consolidated Total,77147


## Price Chart

### Pricing Data  - 1 -  Closing Prices

In [336]:

dfPricingIntermediate1, err = ek.get_data(RIC, 'TR.PriceClose.calcdate;TR.PriceClose',{'sdate':'-5AY','edate':'0d','frq':'D'})
dfPricingIntermediate1


,Instrument,Calc Date,Price Close
0,IBM,2015-03-16,157.08
1,IBM,2015-03-17,156.96
2,IBM,2015-03-18,159.81
3,IBM,2015-03-19,159.81
4,IBM,2015-03-20,162.88
5,IBM,2015-03-23,164.63
6,IBM,2015-03-24,163.00
7,IBM,2015-03-25,159.20
8,IBM,2015-03-26,160.59
9,IBM,2015-03-27,160.40


### Pricing Data - 2 - Dates 

In [335]:
# Intermediate dates to use in the next step
import datetime
from datetime import datetime
from datetime import timedelta

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += timedelta(days=1)
    return days

RIC2 = 'MSFT.O'

dfDatesMSFTIntermediate1, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'0d'})
dfDatesMSFTIntermediate2, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-24AM'})


# number of work days between the two dates + 250
daysBetween = len(workdays((datetime.strptime(dfDatesMSFTIntermediate2.at[0,'Date'],'%Y-%m-%dT%H:%M:%SZ')),
               (datetime.strptime(dfDatesMSFTIntermediate1.at[0,'Date'],'%Y-%m-%dT%H:%M:%SZ')))) + 250
daysBetween
dfDatesMSFTIntermediate4, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-'+str(daysBetween)+'D'})

display(dfDatesMSFTIntermediate1.at[0,'Date'],
dfDatesMSFTIntermediate2.at[0,'Date'],
dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z',
dfDatesMSFTIntermediate4.at[0,'Date'])

if dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z' > dfDatesMSFTIntermediate2.at[0,'Date']:
    laterDate1 = dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z'
else:
    laterDate1 = dfDatesMSFTIntermediate2.at[0,'Date']
    

if dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z' > dfDatesMSFTIntermediate4.at[0,'Date']:
    laterDate2 = dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z'
else:
    laterDate2 = dfDatesMSFTIntermediate4.at[0,'Date']
    
display(laterDate1,laterDate2)

'2020-03-13T00:00:00Z'

'2018-03-16T00:00:00Z'

'2015-03-16T00:00:00Z'

'2017-02-17T00:00:00Z'

'2018-03-16T00:00:00Z'

'2017-02-17T00:00:00Z'

### Pricing Data - 3 - Volumes, Prices, 50 days, 200 days 

In [352]:
dfDtVolPrice, err = ek.get_data(RIC, 'TR.PriceClose.CalcDate,TR.Volume,TR.PriceClose',{'Sdate':'0D','Edate':laterDate2[:10],'Frq':'D'})
dfDtVolPrice                                                          

,Instrument,Calc Date,Volume,Price Close
0,IBM,2020-03-16,10547313,99.08
1,IBM,2020-03-13,12502113,107.95
2,IBM,2020-03-12,12512655,102.81
3,IBM,2020-03-11,8451038,117.97
4,IBM,2020-03-10,11410688,124.77
5,IBM,2020-03-09,10757487,117.81
6,IBM,2020-03-06,8193257,127.73
7,IBM,2020-03-05,5446227,129.55
8,IBM,2020-03-04,4996370,134.22
9,IBM,2020-03-03,7896068,128.90
